- Telemetry Time Series Data (PdM_telemetry.csv): It consists of hourly average of voltage, rotation, pressure, vibration collected from 100 machines for the year 2015.

- Error (PdM_errors.csv): These are errors encountered by the machines while in operating condition. Since, these errors don't shut down the machines, these are not considered as failures. The error date and times are rounded to the closest hour since the telemetry data is collected at an hourly rate.

- Maintenance (PdM_maint.csv): If a component of a machine is replaced, that is captured as a record in this table. Components are replaced under two situations: 1. During the regular scheduled visit, the technician replaced it (Proactive Maintenance) 2. A component breaks down and then the technician does an unscheduled maintenance to replace the component (Reactive Maintenance). This is considered as a failure and corresponding data is captured under Failures. Maintenance data has both 2014 and 2015 records. This data is rounded to the closest hour since the telemetry data is collected at an hourly rate.

- Failures (PdM_failures.csv): Each record represents replacement of a component due to failure. This data is a subset of Maintenance data. This data is rounded to the closest hour since the telemetry data is collected at an hourly rate.

- Metadata of Machines (PdM_Machines.csv): Model type & age of the Machines.

### Import Libs

In [23]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

### Load Data

In [4]:
df_errors = pd.read_csv('data/PdM_errors.csv')
df_errors.head()

,datetime,machineID,errorID
0,2015-01-03 07:00:00,1,error1
1,2015-01-03 20:00:00,1,error3
2,2015-01-04 06:00:00,1,error5
3,2015-01-10 15:00:00,1,error4
4,2015-01-22 10:00:00,1,error4


In [5]:
df_failures = pd.read_csv('data/PdM_failures.csv')
df_failures.head()

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4


In [6]:
df_machines = pd.read_csv('data/PdM_machines.csv')
df_machines.head()

,machineID,model,age
0,1,model3,18
1,2,model4,7
2,3,model3,8
3,4,model3,7
4,5,model3,2


In [7]:
df_telemetry = pd.read_csv('data/PdM_telemetry.csv')
df_telemetry.head()

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [169]:
df_maint = pd.read_csv('data/PdM_maint.csv')
df_maint.head()

,datetime,machineID,comp
0,2014-06-01 06:00:00,1,comp2
1,2014-07-16 06:00:00,1,comp4
2,2014-07-31 06:00:00,1,comp3
3,2014-12-13 06:00:00,1,comp1
4,2015-01-05 06:00:00,1,comp4


### Descriptive Statistics

In [171]:
print('Telemetry - Shape:', df_telemetry.shape)
print('Errors - Shape:', df_errors.shape)
print('Failures - Shape:', df_failures.shape)
print('Machines - Shape:', df_machines.shape)
print('Maintenance - Shape:', df_maint.shape)

Telemetry - Shape: (876100, 6)
Errors - Shape: (3919, 3)
Failures - Shape: (761, 3)
Machines - Shape: (100, 3)
Maintenance - Shape: (3286, 3)


In [10]:
df_telemetry.dtypes

datetime      object
machineID      int64
volt         float64
rotate       float64
pressure     float64
vibration    float64
dtype: object

In [12]:
df_failures.dtypes

datetime     object
machineID     int64
failure      object
dtype: object

In [13]:
df_machines.dtypes

machineID     int64
model        object
age           int64
dtype: object

In [14]:
df_errors.dtypes

datetime     object
machineID     int64
errorID      object
dtype: object

In [172]:
df_maint.dtypes

datetime     object
machineID     int64
comp         object
dtype: object

In [15]:
df_errors.head()

,datetime,machineID,errorID
0,2015-01-03 07:00:00,1,error1
1,2015-01-03 20:00:00,1,error3
2,2015-01-04 06:00:00,1,error5
3,2015-01-10 15:00:00,1,error4
4,2015-01-22 10:00:00,1,error4


In [173]:
# Convert all dates to datetime
df_errors['datetime'] = pd.to_datetime(df_errors['datetime'])
df_failures['datetime'] = pd.to_datetime(df_failures['datetime'])
df_telemetry['datetime'] = pd.to_datetime(df_telemetry['datetime'])
df_maint['datetime'] = pd.to_datetime(df_maint['datetime'])

In [18]:
df_errors.describe()

,datetime,machineID
count,3919,3919.000000
mean,2015-07-01 03:34:18.586374144,51.044654
min,2015-01-01 06:00:00,1.000000
25%,2015-03-31 05:00:00,25.000000
50%,2015-07-01 06:00:00,51.000000
75%,2015-10-01 15:00:00,77.000000
max,2016-01-01 05:00:00,100.000000
std,NaN,28.954988


In [19]:
df_telemetry.describe()

,datetime,machineID,volt,rotate,pressure,vibration
count,876100,876100.000000,876100.000000,876100.000000,876100.000000,876100.000000
mean,2015-07-02 18:00:00,50.500000,170.777736,446.605119,100.858668,40.385007
min,2015-01-01 06:00:00,1.000000,97.333604,138.432075,51.237106,14.877054
25%,2015-04-02 12:00:00,25.750000,160.304927,412.305714,93.498181,36.777299
50%,2015-07-02 18:00:00,50.500000,170.607338,447.558150,100.425559,40.237247
75%,2015-10-02 00:00:00,75.250000,181.004493,482.176600,107.555231,43.784938
max,2016-01-01 06:00:00,100.000000,255.124717,695.020984,185.951998,76.791072
std,NaN,28.866087,15.509114,52.673886,11.048679,5.370361


In [20]:
df_failures.describe()

,datetime,machineID
count,761,761.000000
mean,2015-06-24 12:12:17.976346880,51.911958
min,2015-01-02 03:00:00,1.000000
25%,2015-03-23 06:00:00,24.000000
50%,2015-06-24 06:00:00,51.000000
75%,2015-09-23 06:00:00,79.000000
max,2015-12-31 06:00:00,100.000000
std,NaN,29.515542


In [21]:
df_machines.describe()

,machineID,age
count,100.000000,100.000000
mean,50.500000,11.330000
std,29.011492,5.856974
min,1.000000,0.000000
25%,25.750000,6.750000
50%,50.500000,12.000000
75%,75.250000,16.000000
max,100.000000,20.000000


In [174]:
df_maint.describe()

,datetime,machineID
count,3286,3286.000000
mean,2015-05-30 14:40:36.518563840,50.284236
min,2014-06-01 06:00:00,1.000000
25%,2015-03-03 06:00:00,25.250000
50%,2015-06-13 06:00:00,50.000000
75%,2015-09-18 00:00:00,75.000000
max,2016-01-01 06:00:00,100.000000
std,NaN,28.914478


- The dataset contains 100 different machines, varying age from 0 to 20 years in service.
- The dataset contains 761 registered failures
- The dataset contains 3919 failures that didn't breakdown the machines.
- Sensors output are averaged hourly.
- There are register for maintenance from 2014 to 2016, summing 3286 replacements.

### Feature Engineering

First approach will be without using time-series.

Related to the sensors:
- Time window - need to test the size:
    - Average of each sensors OK
    - Std of each sensors OK
    - Max and Min of each sensors OK
    - Number of times that data exceeds a threshold related to the average of gold machines (without errors) OK
    - Average/Std/Min/Max of each sensor for different shifts (morning, afternoon, night, dawn)
- Count of the number of times a certain error code or component replacement occurred within a certain time period.  OK 
- Total uptime and downtime of the machine over a certain time period.
- Average time between maintenance activities.
- Count the number of maintenance activities performed on a machine.
- Age of the machine
- Days since last replacement
- Days since last failure OK
- Days since last errors OK

The structure is a time series, but we are not using it as input.

Datetime (day) | MachineID | Characteristics
- Day 1 | Machine 1 | Characteristics
- Day 1 | Machine 2 | Characteristics
- Day 2 | Machine 1 | Characteristics
- Day 2 | Machine 2 | Characteristics

The machines will repeat for each day, the idea is to create a type of survival analysis, which means that each day, the probability to failure in the next X days should increase untill the day of failure.


#### Creating Features

In [74]:
gold_machines = list(set([i for i in range(1, 101)]) - set(df_failures['machineID'].unique()))

In [89]:
df_gold = df_telemetry[df_telemetry['machineID'].isin(gold_machines)]
df_gold = df_gold.mean().reset_index().T
df_gold.columns = df_gold.loc['index']
df_gold = df_gold.iloc[1:]
df_gold

index,datetime,machineID,volt,rotate,pressure,vibration
0,2015-07-02 17:59:59.999999744,41.5,170.478939,448.043332,100.595992,40.322392


In [47]:
def select_window(df, end, window):
    end = pd.to_datetime(end, format = '%Y-%m-%d')
    start = end - timedelta(days = window)
    return df[(df['datetime'] >= start) & (df['datetime'] < end)]

In [40]:
day = '2015-02-01'

In [56]:
df_selected

,datetime,machineID,volt,rotate,pressure,vibration
714,2015-01-31 00:00:00,1,205.367261,377.513739,104.433511,44.749329
715,2015-01-31 01:00:00,1,167.111760,470.266601,94.309670,43.490986
716,2015-01-31 02:00:00,1,170.099290,487.098625,86.477943,40.371791
717,2015-01-31 03:00:00,1,169.181810,426.012442,103.358262,44.590389
718,2015-01-31 04:00:00,1,161.476897,435.458549,103.672254,31.283585
...,...,...,...,...,...,...
868072,2015-01-31 19:00:00,100,155.905844,481.602194,108.300089,40.219653
868073,2015-01-31 20:00:00,100,179.752914,441.626479,92.479003,29.884819
868074,2015-01-31 21:00:00,100,185.538424,451.954396,106.218228,40.254770
868075,2015-01-31 22:00:00,100,173.924477,449.205194,105.631238,44.356578


In [110]:
df_selected = select_window(df_telemetry, day, 1)
df_summary = pd.DataFrame()

df_summary['machineID'] = [i for i in range(1, 101)]
df_summary['dtRef'] = day
for col in ['volt', 'pressure', 'rotate', 'vibration']:

    df1 = df_selected.groupby('machineID')[col].mean().reset_index().rename(columns = {col: col + '_mean_24h'})
    df1[col + '_std_24h'] = df_selected.groupby('machineID')[col].std().values.tolist()
    df1[col + '_min_24h'] = df_selected.groupby('machineID')[col].min().values.tolist()
    df1[col + '_max_24h'] = df_selected.groupby('machineID')[col].max().values.tolist()
    df1[col + '_max_rate_24h'] = df1[col + '_max_24h'].apply(lambda x: x / df_gold[col])
    df_summary = pd.merge(df_summary, df1, on = 'machineID', how = 'left')

df_summary

,machineID,dtRef,volt_mean_24h,volt_std_24h,volt_min_24h,volt_max_24h,volt_max_rate_24h,pressure_mean_24h,pressure_std_24h,pressure_min_24h,...,rotate_mean_24h,rotate_std_24h,rotate_min_24h,rotate_max_24h,rotate_max_rate_24h,vibration_mean_24h,vibration_std_24h,vibration_min_24h,vibration_max_24h,vibration_max_rate_24h
0,1,2015-02-01,173.160661,14.556373,152.963972,205.367261,1.204649,96.988903,11.631080,68.823290,...,443.410469,48.772770,346.875533,530.204506,1.183378,40.319582,4.125902,31.283585,49.689049,1.232294
1,2,2015-02-01,167.342382,12.738052,141.511811,200.388192,1.175443,99.344102,10.694141,72.021538,...,439.747748,39.371950,381.918288,502.804031,1.122222,38.870068,5.128843,31.639294,50.320680,1.247959
2,3,2015-02-01,165.210771,10.653568,146.062385,185.288988,1.086873,100.854992,12.200322,71.999966,...,453.890722,53.202075,344.183443,533.594455,1.190944,40.655328,5.449210,31.912944,52.613634,1.304824
3,4,2015-02-01,168.335624,11.779999,147.554097,187.429637,1.099430,100.222023,9.291577,74.491418,...,446.937501,52.806669,364.193241,569.768066,1.271681,42.128809,4.658741,32.933670,48.886217,1.212384
4,5,2015-02-01,171.499839,15.707603,147.664771,197.585615,1.159003,99.890466,8.710010,85.991750,...,457.642766,44.847421,362.615765,540.083749,1.205427,40.578897,3.642439,33.839365,45.635725,1.131771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2015-02-01,172.352787,15.738535,141.817403,206.876519,1.213502,101.477634,10.442714,84.171583,...,449.176652,42.898987,376.314750,532.994860,1.189606,38.404873,5.211213,26.037747,46.633189,1.156508
96,97,2015-02-01,171.088624,17.192708,137.325145,205.160912,1.203438,100.723193,12.470710,78.655386,...,452.205342,47.736880,370.553662,538.869131,1.202717,39.751038,7.297631,25.149462,52.246644,1.295723
97,98,2015-02-01,166.545662,15.697937,141.537790,204.675066,1.200589,101.287104,7.805200,82.376195,...,440.922860,59.024787,320.367696,568.824780,1.269575,38.293645,4.883602,25.898924,47.768106,1.184655
98,99,2015-02-01,186.347286,18.814198,158.254887,235.001645,1.378479,98.238925,10.012305,78.619850,...,464.167976,51.475348,372.279145,594.447460,1.326763,38.888779,4.210216,29.091847,47.031533,1.166387


In [118]:
thr = 0.3
df_count = pd.DataFrame()
df_count['machineID'] = [i for i in range(1, 101)]
df_count['dfRef'] = day
for col in ['volt', 'pressure', 'rotate', 'vibration']:
    df_selected = select_window(df_telemetry, day, 1)
    df1 = df_selected[df_selected[col] > df_gold[col].values[0] * (1 + thr)].groupby('machineID')['datetime'].count().reset_index().rename(columns = {'datetime': col + '_count_exceed_thr_24h'})
    df_count = pd.merge(df_count, df1, on = 'machineID', how = 'left').fillna(0)
df_count


,machineID,datetime,volt_count_exceed_thr_24h,pressure_count_exceed_thr_24h,rotate_count_exceed_thr_24h,vibration_count_exceed_thr_24h
0,1,2015-02-01,0.0,0.0,0.0,0.0
1,2,2015-02-01,0.0,0.0,0.0,0.0
2,3,2015-02-01,0.0,0.0,0.0,1.0
3,4,2015-02-01,0.0,0.0,0.0,0.0
4,5,2015-02-01,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
95,96,2015-02-01,0.0,0.0,0.0,0.0
96,97,2015-02-01,0.0,1.0,0.0,0.0
97,98,2015-02-01,0.0,0.0,0.0,0.0
98,99,2015-02-01,1.0,0.0,1.0,0.0


In [140]:
df1 = select_window(df_errors, day, 15)
df_count_errors = pd.DataFrame()
df_count_errors['machineID'] = [i for i in range(1, 101)]
df_count_errors['dtRef'] = day
for error in df_errors['errorID'].unique():
    df2 = df1[df1['errorID'] == error].groupby('machineID')['datetime'].count().reset_index().rename(columns = {'datetime': 'count_' + error + '_last15days'})

    df_count_errors = pd.merge(df_count_errors, df2, on = 'machineID', how = 'left').fillna(0)

df_count_errors

,machineID,datetime,count_error1_last15days,count_error3_last15days,count_error5_last15days,count_error4_last15days,count_error2_last15days
0,1,2015-02-01,1.0,0.0,0.0,2.0,0.0
1,2,2015-02-01,0.0,0.0,0.0,0.0,0.0
2,3,2015-02-01,0.0,0.0,0.0,1.0,0.0
3,4,2015-02-01,0.0,0.0,0.0,0.0,0.0
4,5,2015-02-01,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
95,96,2015-02-01,0.0,0.0,0.0,0.0,0.0
96,97,2015-02-01,0.0,0.0,1.0,4.0,1.0
97,98,2015-02-01,1.0,0.0,0.0,1.0,0.0
98,99,2015-02-01,0.0,0.0,1.0,0.0,3.0


In [155]:
df_since_last_failure = pd.DataFrame()
df_since_last_failure['machineID'] = [i for i in range(1, 101)]
df_since_last_failure['dtRef'] = day
df_selected_failures = df_failures[df_failures['datetime'] <= day]
df_selected_failures = df_selected_failures.groupby('machineID')['datetime'].first().reset_index()
df_selected_failures['days_since_last_failure'] = df_selected_failures['datetime'].apply(lambda x: (pd.to_datetime(day, format = '%Y-%m-%d') - x) / pd.Timedelta(days=1))
df_since_last_failure = pd.merge(df_since_last_failure,df_selected_failures.drop('datetime', axis = 1), on = 'machineID', how = 'left')
df_since_last_failure

,machineID,dtRef,days_since_last_failure
0,1,2015-02-01,26.750
1,2,2015-02-01,NaN
2,3,2015-02-01,24.750
3,4,2015-02-01,14.750
4,5,2015-02-01,22.750
...,...,...,...
95,96,2015-02-01,NaN
96,97,2015-02-01,7.750
97,98,2015-02-01,1.750
98,99,2015-02-01,29.875


In [159]:
df_since_last_errors = pd.DataFrame()
df_since_last_errors['machineID'] = [i for i in range(1, 101)]
df_since_last_errors['dtRef'] = day
df_selected_errors = df_errors[df_errors['datetime'] <= day]
df_selected_errors = df_selected_errors.groupby('machineID')['datetime'].last().reset_index()
df_selected_errors['days_since_last_error'] = df_selected_errors['datetime'].apply(lambda x: (pd.to_datetime(day, format = '%Y-%m-%d') - x) / pd.Timedelta(days=1))
df_since_last_errors = pd.merge(df_since_last_errors,df_selected_errors.drop('datetime', axis = 1), on = 'machineID', how = 'left')
df_since_last_errors

,machineID,dtRef,days_since_last_error
0,1,2015-02-01,4.833333
1,2,2015-02-01,19.416667
2,3,2015-02-01,3.291667
3,4,2015-02-01,15.750000
4,5,2015-02-01,0.208333
...,...,...,...
95,96,2015-02-01,26.416667
96,97,2015-02-01,5.625000
97,98,2015-02-01,2.750000
98,99,2015-02-01,1.583333


In [177]:
df_since_last_rep = pd.DataFrame()
df_since_last_rep['machineID'] = [i for i in range(1, 101)]
df_since_last_rep['dtRef'] = day
df_selected_rep = df_maint[df_maint['datetime'] <= day]
df_selected_rep = df_selected_rep.groupby('machineID')['datetime'].last().reset_index()
df_selected_rep['days_since_last_rep'] = df_selected_rep['datetime'].apply(lambda x: (pd.to_datetime(day, format = '%Y-%m-%d') - x) / pd.Timedelta(days=1))
df_since_last_rep = pd.merge(df_since_last_rep,df_selected_rep.drop('datetime', axis = 1), on = 'machineID', how = 'left')
df_since_last_rep

,machineID,dtRef,days_since_last_rep
0,1,2015-02-01,11.75
1,2,2015-02-01,13.75
2,3,2015-02-01,9.75
3,4,2015-02-01,14.75
4,5,2015-02-01,7.75
...,...,...,...
95,96,2015-02-01,13.75
96,97,2015-02-01,7.75
97,98,2015-02-01,1.75
98,99,2015-02-01,13.75


In [166]:
df_charact_machines = df_machines.copy()
df_charact_machines['dtRef'] = day
df_charact_machines

,machineID,model,age,dtRef
0,1,model3,18,2015-02-01
1,2,model4,7,2015-02-01
2,3,model3,8,2015-02-01
3,4,model3,7,2015-02-01
4,5,model3,2,2015-02-01
...,...,...,...,...
95,96,model2,10,2015-02-01
96,97,model2,14,2015-02-01
97,98,model2,20,2015-02-01
98,99,model1,14,2015-02-01


#### Creating the Target

In [188]:
next_days = 7
print(day)
df_target = pd.DataFrame()
df_target['machineID'] = [i for i in range(1, 101)]
df_target['dtRef'] = day
df_failures_select = df_failures[(df_failures['datetime'] > pd.to_datetime(day, format = '%Y-%m-%d')) & (df_failures['datetime'] <= pd.to_datetime(day, format = '%Y-%m-%d') + pd.Timedelta(days=next_days))]
df_failures_count = df_failures_select.groupby('machineID')['datetime'].count().reset_index().rename(columns = {'datetime': 'count_failures'})
df_failures_count['target'] = df_failures_count['count_failures'].apply(lambda x: 1 if x > 0 else 0)
df_target = pd.merge(df_target, df_failures_count[['machineID', 'target']], on = 'machineID', how = 'left').fillna(0)
df_target

2015-02-01


,machineID,dtRef,target
0,1,2015-02-01,0.0
1,2,2015-02-01,0.0
2,3,2015-02-01,1.0
3,4,2015-02-01,0.0
4,5,2015-02-01,0.0
...,...,...,...
95,96,2015-02-01,0.0
96,97,2015-02-01,0.0
97,98,2015-02-01,0.0
98,99,2015-02-01,1.0


### Exploratory Data Analysis

### Pre-processing

### Modeling